###### Spark Transformations and actions results in execution plan. 
###### This execution plan finally execued by executors.

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

from lib.logger import Log4j
from lib.utils import get_spark_app_config
from lib.utils import load_survey_df

In [2]:
conf = get_spark_app_config()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [7]:
spark

In [3]:
logger = Log4j(spark)
logger.info("Starting HelloSpark")

We are doing three thing here
1. Reading csv and infereing schema - result is dataframe
2. Applying Chain of Transformations - result is another dataframe
3. we are using action show()

In [4]:
survey_df = load_survey_df(spark,"data/sample.csv")

In [5]:
count_df = survey_df.where("Age>40").select("Age","Gender","Country","state").groupBy("Country").count()

In [6]:
count_df.show()

+-------------+-----+
|      Country|count|
+-------------+-----+
|United States|    2|
+-------------+-----+



In [ ]:
spark.stop()

# Modular Structure

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

from lib.logger import Log4j
from lib.utils import get_spark_app_config,load_survey_df,count_by_country

In [2]:
conf = get_spark_app_config()
spark = SparkSession.builder.config(conf=conf).getOrCreate()
logger = Log4j(spark)
logger.info("Starting HelloSpark")
survey_df = load_survey_df(spark,"data/sample.csv")

Based on this example we need to understand internal execution of plan.
now csv file that we are reading and when it loaded as dataframe it have just one partition.
Not good for learning real internal behaviour.
###### Lets Simulate Multiple partions

In [3]:
partitioned_df = survey_df.repartition(2)

we partitioned survey_df into two partitions. will use for further transformations.

now we are appluying groupBy as wide transformation so it cause internal partitions.

we created two forced partitions on dataframe by applying reparttion transformation but we cannot have control over parttion caused internally by suffle sort to handle wide transformations.

to control this behaviour will use configuration. spark.sql.shuffle.partition property will set number of partition. add this property value in spark.conf.

Replace show() with collect() action and write info to logger
collect action returns the dataframe as python list. show method is utilty function to print the dataframe. 

In [4]:
count_df = count_by_country(partitioned_df)
logger.info(count_df.collect())

In [5]:
input("Press Enter")

Press Enter


''

We are executing this execution plan locally and look at spark UI to get details. Spark UI only available during life of an application. so i am going to ask user input here in above code cell. so till we eneter somethinf application is live and will visit SPARK UI to get execution plan.
But remember to remove this line later. because this line is forlocal debugging not for production.

In [ ]:
spark.stop()